<a href="https://colab.research.google.com/github/tatsubo777/machine_learning_study/blob/main/%E7%AC%AC8%E7%AB%A0_%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92%E3%81%AE%E9%81%A9%E7%94%A81_%E6%84%9F%E6%83%85%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

自然言語処理の一分野である感情（センチメント）分析を取り上げる



*   テキストデータのクレンジングと準備
*   テキスト文書からの特徴量ベクトルの構築
*   映画レビューを肯定的な意見と否定的な意見に分類する機械学習モデルの構築
*   文書コレクションからのトピックカテゴリの推定


**8.1 IMDbの映画レビューデータセットのテキスト処理**

感情分析は文書の極性を分析することに関連している。



**8.1.1映画レビューデータセットを取得する**

In [ ]:
import os
import sys
import tarfile
import time


source = 'http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
target = 'aclImdb_v1.tar.gz'


def reporthook(count, block_size, total_size):
    global start_time
    if count == 0:
        start_time = time.time()
        return
    duration = time.time() - start_time
    progress_size = int(count * block_size)
    speed = progress_size / (1024.**2 * duration)
    percent = count * block_size * 100. / total_size
    sys.stdout.write("\r%d%% | %d MB | %.2f MB/s | %d sec elapsed" %
                    (percent, progress_size / (1024.**2), speed, duration))
    sys.stdout.flush()


if not os.path.isdir('aclImdb') and not os.path.isfile('aclImdb_v1.tar.gz'):
    
    if (sys.version_info < (3, 0)):
        import urllib
        urllib.urlretrieve(source, target, reporthook)
    
    else:
        import urllib.request
        urllib.request.urlretrieve(source, target, reporthook)

100% | 80 MB | 2.39 MB/s | 33 sec elapsed

In [ ]:
if not os.path.isdir('aclImdb'):

    with tarfile.open(target, 'r:gz') as tar:
        tar.extractall()

**8.1.2映画レビューデータセットをより便利なフォーマットに変換する**

 pyprind =進行状況と完了までの推定時間を確認するにはpyprindを活用する

In [ ]:
!pip install pyprind

In [ ]:
import pyprind
import pandas as pd
import os

basepath = 'aclImdb'

labels = {'pos': 1, 'neg': 0}
pbar = pyprind.ProgBar(50000)
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file), 
                      'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], 
                           ignore_index=True)
            pbar.update()
df.columns = ['review', 'sentiment']



0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:38


In [ ]:
import numpy as np

np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))

In [ ]:
df.to_csv('movie_data.csv',index=False,encoding='utf-8')

In [ ]:
df.head(3)

,review,sentiment
0,I went and saw this movie last night after bei...,1
1,Actor turned director Bill Paxton follows up h...,1
2,As a recreational golfer with some knowledge o...,1


In [ ]:
import pandas as  pd

df = pd.read_csv('movie_data.csv',encoding='utf-8')
df.head(3)

,review,sentiment
0,I went and saw this movie last night after bei...,1
1,Actor turned director Bill Paxton follows up h...,1
2,As a recreational golfer with some knowledge o...,1


In [ ]:
df.shape

(50000, 2)

**8.2 BoW モデルの紹介**

機械学習アルゴリズムに渡す前に数値に変換する必要がある。

ここではテキストを数値の特徴量ベクトルとして表現できるBoWモデルを紹介する。



1.   文書の集合全体から、例えば単語という一意なトークンからなる語彙を作成する
2.   各文書での各単語の出現回数を含んだ特徴量ベクトルを構築する。



**8.2.1 単語を特徴量ベクトルに変換する**

BoWモデルを構築するにはscikit-learnのCountVectorizerを使えば良い

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
docs = np.array([
                 'The sun is shining',
                 'The weather is sweet',
                 'The sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)

基本概念をよく理解できるよう、語彙の中身を出力してみよう

In [ ]:
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


語彙はディクショナリに格納されている。

In [ ]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


**8.2.2 TF-IDFを使って関連性を評価する**

TF（単語の出現頻度）とIDF（逆文書頻度）の積として定義できる。

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True,
                         norm='l2',
                         smooth_idf=True)
print(tfidf.fit_transform(count.fit_transform(docs))
.toarray())

[[0.         0.43370786 0.         0.55847784 0.55847784 0.
  0.43370786 0.         0.        ]
 [0.         0.43370786 0.         0.         0.         0.55847784
  0.43370786 0.         0.55847784]
 [0.50238645 0.44507629 0.50238645 0.19103892 0.19103892 0.19103892
  0.29671753 0.25119322 0.19103892]]
